In [ ]:
!pip install datasets pandas

In [ ]:
# Load PubMedQA DB
from datasets import load_dataset
dataset = load_dataset("qiaojin/PubMedQA", 'pqa_labeled', split="train")
#dataset[:5]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!mkdir -p ~/.kaggle
!cp /content/drive/MyDrive/Kaggle_API/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d moaaztameer/medqa-usmle

Dataset URL: https://www.kaggle.com/datasets/moaaztameer/medqa-usmle
License(s): other
  0% 0.00/99.0M [00:00<?, ?B/s]
100% 99.0M/99.0M [00:00<00:00, 1.16GB/s]


In [ ]:
!unzip /content/medqa-usmle.zip

Archive:  /content/medqa-usmle.zip
  inflating: MedQA-USMLE/questions/Mainland/4_options/dev.jsonl  
  inflating: MedQA-USMLE/questions/Mainland/4_options/test.jsonl  
  inflating: MedQA-USMLE/questions/Mainland/4_options/train.jsonl  
  inflating: MedQA-USMLE/questions/Mainland/chinese_qbank.jsonl  
  inflating: MedQA-USMLE/questions/Mainland/dev.jsonl  
  inflating: MedQA-USMLE/questions/Mainland/test.jsonl  
  inflating: MedQA-USMLE/questions/Mainland/train.jsonl  
  inflating: MedQA-USMLE/questions/Taiwan/dev.jsonl  
  inflating: MedQA-USMLE/questions/Taiwan/metamap/dev/tw_dev.jsonl  
  inflating: MedQA-USMLE/questions/Taiwan/metamap/test/tw_test.jsonl  
  inflating: MedQA-USMLE/questions/Taiwan/metamap/train/tw_train.jsonl  
  inflating: MedQA-USMLE/questions/Taiwan/taiwanese_qbank.jsonl  
  inflating: MedQA-USMLE/questions/Taiwan/test.jsonl  
  inflating: MedQA-USMLE/questions/Taiwan/train.jsonl  
  inflating: MedQA-USMLE/questions/Taiwan/tw_translated_jsonl/en/dev-2en.jsonl  
  

In [ ]:
# Load MedQA DB

import json

data_path = "/content/MedQA-USMLE/questions/US/train.jsonl"

data_list = []
with open(data_path, 'r') as f:
    for line in f:
        data_list.append(json.loads(line))
medqa_df = pd.DataFrame(data_list)
print(medqa_df.head())

                                            question  \
0  A 23-year-old pregnant woman at 22 weeks gesta...   
1  A 3-month-old baby died suddenly at night whil...   
2  A mother brings her 3-week-old infant to the p...   
3  A pulmonary autopsy specimen from a 58-year-ol...   
4  A 20-year-old woman presents with menorrhagia ...   

                                              answer  \
0                                     Nitrofurantoin   
1  Placing the infant in a supine position on a f...   
2       Abnormal migration of ventral pancreatic bud   
3                                    Thromboembolism   
4                             Von Willebrand disease   

                                             options meta_info answer_idx  
0  {'A': 'Ampicillin', 'B': 'Ceftriaxone', 'C': '...   step2&3          E  
1  {'A': 'Placing the infant in a supine position...   step2&3          A  
2  {'A': 'Abnormal migration of ventral pancreati...     step1          A  
3  {'A': 'Thromboembol

# Task
Modify the selected empty cell to pair and format the "dataset" and "medqa_df" datasets for fine-tuning, following the recommended fine-tuning dataset format.

## Define the pairing strategy

### Subtask:
Based on the dataset structures, determine how to create pairs of data that will be used for fine-tuning. This might involve extracting question-answer pairs, or creating prompt-completion pairs from the available information.


**Reasoning**:
Analyze the structures of both datasets to identify question and answer fields and outline a strategy for pairing them for fine-tuning.



In [ ]:
print("PubMedQA dataset keys:", dataset.features.keys())
print("MedQA-USMLE dataframe columns:", medqa_df.columns)

# Strategy for pairing:
# PubMedQA: Use 'question' as the prompt and the 'long_answer' within 'context' as the completion.
# MedQA-USMLE: Use 'question' as the prompt and 'answer' as the completion.
# Both datasets will be converted into a list of dictionaries with 'prompt' and 'completion' keys for fine-tuning.

PubMedQA dataset keys: dict_keys(['pubid', 'question', 'context', 'long_answer', 'final_decision'])
MedQA-USMLE dataframe columns: Index(['question', 'answer', 'options', 'meta_info', 'answer_idx'], dtype='object')


## Choose a fine-tuning format

### Subtask:
Select a suitable format for the fine-tuned dataset. Common formats include a list of dictionaries with 'prompt' and 'completion' keys, or a conversational format with 'role' and 'content' for user and assistant turns.


## Format the paired data

### Subtask:
Transform the paired data into the chosen fine-tuning format. This will likely involve iterating through the datasets, extracting the relevant information, and structuring it into the desired output format.


In [ ]:
fine_tuning_data = []

# Process PubMedQA dataset
for item in dataset:
    question = item['question']
    long_answer = item['long_answer']
    fine_tuning_data.append({
        'prompt': question,
        'completion': long_answer
    })

# Process MedQA-USMLE dataframe
for index, row in medqa_df.iterrows():
    question = row['question']
    answer = row['answer']
    fine_tuning_data.append({
        'prompt': question,
        'completion': answer
    })

print(f"Created {len(fine_tuning_data)} prompt-completion pairs.")
print("First 5 entries of fine_tuning_data:")
display(fine_tuning_data[:5])

Created 11178 prompt-completion pairs.
First 5 entries of fine_tuning_data:


[{'prompt': 'Do mitochondria play a role in remodelling lace plant leaves during programmed cell death?',
  'completion': 'Results depicted mitochondrial dynamics in vivo as PCD progresses within the lace plant, and highlight the correlation of this organelle with other organelles during developmental PCD. To the best of our knowledge, this is the first report of mitochondria and chloroplasts moving on transvacuolar strands to form a ring structure surrounding the nucleus during developmental PCD. Also, for the first time, we have shown the feasibility for the use of CsA in a whole plant system. Overall, our findings implicate the mitochondria as playing a critical and early role in developmentally regulated PCD in the lace plant.'},
 {'prompt': 'Landolt C and snellen e acuity: differences in strabismus amblyopia?',
  'completion': 'Using the charts described, there was only a slight overestimation of visual acuity by the Snellen E compared to the Landolt C, even in strabismus amblyopi

## Combine the datasets (optional)

### Subtask:
Combine the formatted data from both the PubMedQA and MedQA-USMLE datasets into a single dataset for fine-tuning.


## Save the formatted dataset

### Subtask:
Save the final formatted dataset in a suitable file format (e.g., JSONL) that can be used for fine-tuning.


**Reasoning**:
Save the combined and formatted data to a JSONL file.



In [ ]:
import json

output_file_path = 'fine_tuning_data.jsonl'

with open(output_file_path, 'w') as f:
    for item in fine_tuning_data:
        f.write(json.dumps(item) + '\n')

print(f"Saved fine-tuning data to {output_file_path}")

Saved fine-tuning data to fine_tuning_data.jsonl


In [ ]:
# Model Initialize (consider limited resource)

from huggingface_hub import login
login() # Call login without arguments to prompt for token

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch # Import torch

# model_name = "TheBloke/llama-2-7b-chat-hf"  # 可依資源換小模型 - This model name caused a previous error.
# model_name = "gpt2" # Changed to a causal language model example.
model_name = "distilgpt2" # Changed to a smaller causal language model for testing.

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

# 低資源加速
from peft import LoraConfig, get_peft_model

# Define the BitsAndBytesConfig for 8-bit quantization
# Note: Ensure bitsandbytes is correctly installed and working for this to apply.
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_8bit_compute_dtype=torch.float16, # Use torch.float16
    bnb_8bit_quant_type="int8", # Changed to a valid value
    bnb_8bit_use_double_quant=False,
)


model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",  # 自動分配 GPU/CPU
    # Remove quantization_config for now to bypass bitsandbytes issues
    # quantization_config=bnb_config, # Use the new quantization_config argument
)

# LoRA 設定
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["c_attn"],  # Changed target modules for distilgpt2
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

# Print model summary after applying PEFT
model.print_trainable_parameters()

trainable params: 147,456 || all params: 82,060,032 || trainable%: 0.1797


/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/layer.py:2174: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [ ]:
def tokenize(example):
    # Full text = prompt + completion
    text = example["prompt"] + " " + example["completion"]
    tokens = tokenizer(
        text,
        truncation=True,
        padding="max_length",
        max_length=512
    )

    # Create labels
    labels = tokens["input_ids"].copy()

    # Mask out prompt part so loss is only on completion
    prompt_len = len(tokenizer(example["prompt"])["input_ids"])
    labels[:prompt_len] = [-100] * prompt_len

    tokens["labels"] = labels
    return tokens

tokenized_dataset = dataset.map(tokenize)

Map:   0%|          | 0/11178 [00:00<?, ? examples/s]

# Task
Split the dataset into training and testing sets, apply a single tokenization function to both splits, initialize a `Trainer` with the tokenized training data, and start the training process.

## Split the dataset

### Subtask:
Split the combined dataset into training and testing sets, and then apply a single tokenization function to both splits.


**Reasoning**:
Split the combined dataset into training and testing sets and then apply the tokenize function to both splits.



In [ ]:
# Split the dataset into training and testing sets
train_test_split = dataset.train_test_split(test_size=0.1)

# Apply the tokenize function to the training and testing splits
tokenized_train_dataset = train_test_split['train'].map(tokenize)
tokenized_test_dataset = train_test_split['test'].map(tokenize)

print(f"Training dataset size: {len(tokenized_train_dataset)}")
print(f"Testing dataset size: {len(tokenized_test_dataset)}")

Map:   0%|          | 0/10060 [00:00<?, ? examples/s]

Map:   0%|          | 0/1118 [00:00<?, ? examples/s]

Training dataset size: 10060
Testing dataset size: 1118


## Initialize the trainer

### Subtask:
Initialize the `Trainer` with the tokenized training dataset and the previously defined training arguments.


**Reasoning**:
Initialize the Trainer with the model, training arguments, and the tokenized training dataset.



In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./fine_tuned_model",
    report_to="none",
    logging_steps=10,
    num_train_epochs=3,
    per_device_train_batch_size=1,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
)

## Start training

### Subtask:
Start the fine-tuning process using the initialized `Trainer` object and save the fine-tuned model after training is complete.


**Reasoning**:
The previous training attempt failed because the model was not returning a loss. This often happens when the tokenized data is not correctly structured with labels for causal language modeling. The `tokenize` function was updated in a subsequent cell to correctly create labels and mask the prompt part. The trainer object needs to be re-initialized with the `tokenized_dataset` which contains the correctly formatted labels, and then the training can be started.



In [ ]:
# Re-initialize the trainer with the correctly tokenized dataset (which now includes labels)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset, # Use the tokenized_dataset with labels
)

# Start the training process
trainer.train()

# Save the fine-tuned model
trainer.save_model("./fine_tuned_model")

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
10,11.180500
20,10.950700
30,10.600400
40,10.826700
50,10.694300
60,10.325400
70,10.122700
80,9.830500
90,9.357500
100,9.022100


Step,Training Loss
10,11.180500
20,10.950700
30,10.600400
40,10.826700
50,10.694300
60,10.325400
70,10.122700
80,9.830500
90,9.357500
100,9.022100


ValueError: Expected input batch_size (512) to match target batch_size (874).